# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 5 2023 10:31AM,254332,12,8993306,HealthCaps LLC,Released
1,Jan 5 2023 10:22AM,254331,19,451750821,"GUSA Granules USA, Inc.",Released
2,Jan 5 2023 10:11AM,254330,10,8993298,"Citieffe, Inc.",Released
3,Jan 5 2023 9:01AM,254327,20,8992764,"Adare Pharmaceuticals, Inc.",Released
4,Jan 5 2023 8:54AM,254324,19,60017902,"GUSA Granules USA, Inc.",Executing
5,Jan 5 2023 8:44AM,254321,16,8990749,Sartorius Bioprocess Solutions,Released
6,Jan 5 2023 8:30AM,254301,10,8990139-SD,Eywa Pharma Inc.,Executing
7,Jan 5 2023 8:01AM,254319,19,ADV80010055,"AdvaGen Pharma, Ltd",Released
8,Jan 5 2023 8:01AM,254319,19,ADV80010056,"AdvaGen Pharma, Ltd",Released
9,Jan 5 2023 8:01AM,254319,19,ADV80010057,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
10,254324,Executing,1
11,254327,Released,1
12,254330,Released,1
13,254331,Released,1
14,254332,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
254324,1.0,NaN
254327,NaN,1.0
254330,NaN,1.0
254331,NaN,1.0
254332,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
254265,0.0,1.0
254270,0.0,1.0
254282,0.0,14.0
254301,1.0,0.0
254302,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
254265,0,1
254270,0,1
254282,0,14
254301,1,0
254302,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,254265,0,1
1,254270,0,1
2,254282,0,14
3,254301,1,0
4,254302,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,254265,,1
1,254270,,1
2,254282,,14
3,254301,1,
4,254302,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 5 2023 10:31AM,254332,12,HealthCaps LLC
1,Jan 5 2023 10:22AM,254331,19,"GUSA Granules USA, Inc."
2,Jan 5 2023 10:11AM,254330,10,"Citieffe, Inc."
3,Jan 5 2023 9:01AM,254327,20,"Adare Pharmaceuticals, Inc."
4,Jan 5 2023 8:54AM,254324,19,"GUSA Granules USA, Inc."
5,Jan 5 2023 8:44AM,254321,16,Sartorius Bioprocess Solutions
6,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.
7,Jan 5 2023 8:01AM,254319,19,"AdvaGen Pharma, Ltd"
57,Jan 4 2023 4:43PM,254312,15,"Person & Covey, Inc."
93,Jan 4 2023 3:56PM,254307,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 5 2023 10:31AM,254332,12,HealthCaps LLC,,1
1,Jan 5 2023 10:22AM,254331,19,"GUSA Granules USA, Inc.",,1
2,Jan 5 2023 10:11AM,254330,10,"Citieffe, Inc.",,1
3,Jan 5 2023 9:01AM,254327,20,"Adare Pharmaceuticals, Inc.",,1
4,Jan 5 2023 8:54AM,254324,19,"GUSA Granules USA, Inc.",1,
5,Jan 5 2023 8:44AM,254321,16,Sartorius Bioprocess Solutions,,1
6,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,1,
7,Jan 5 2023 8:01AM,254319,19,"AdvaGen Pharma, Ltd",,50
8,Jan 4 2023 4:43PM,254312,15,"Person & Covey, Inc.",,36
9,Jan 4 2023 3:56PM,254307,19,"AdvaGen Pharma, Ltd",,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 5 2023 10:31AM,254332,12,HealthCaps LLC,1,
1,Jan 5 2023 10:22AM,254331,19,"GUSA Granules USA, Inc.",1,
2,Jan 5 2023 10:11AM,254330,10,"Citieffe, Inc.",1,
3,Jan 5 2023 9:01AM,254327,20,"Adare Pharmaceuticals, Inc.",1,
4,Jan 5 2023 8:54AM,254324,19,"GUSA Granules USA, Inc.",,1
5,Jan 5 2023 8:44AM,254321,16,Sartorius Bioprocess Solutions,1,
6,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,,1
7,Jan 5 2023 8:01AM,254319,19,"AdvaGen Pharma, Ltd",50,
8,Jan 4 2023 4:43PM,254312,15,"Person & Covey, Inc.",36,
9,Jan 4 2023 3:56PM,254307,19,"AdvaGen Pharma, Ltd",6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 5 2023 10:31AM,254332,12,HealthCaps LLC,1,
1,Jan 5 2023 10:22AM,254331,19,"GUSA Granules USA, Inc.",1,
2,Jan 5 2023 10:11AM,254330,10,"Citieffe, Inc.",1,
3,Jan 5 2023 9:01AM,254327,20,"Adare Pharmaceuticals, Inc.",1,
4,Jan 5 2023 8:54AM,254324,19,"GUSA Granules USA, Inc.",,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 5 2023 10:31AM,254332,12,HealthCaps LLC,1.0,NaN
1,Jan 5 2023 10:22AM,254331,19,"GUSA Granules USA, Inc.",1.0,NaN
2,Jan 5 2023 10:11AM,254330,10,"Citieffe, Inc.",1.0,NaN
3,Jan 5 2023 9:01AM,254327,20,"Adare Pharmaceuticals, Inc.",1.0,NaN
4,Jan 5 2023 8:54AM,254324,19,"GUSA Granules USA, Inc.",NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 5 2023 10:31AM,254332,12,HealthCaps LLC,1.0,0.0
1,Jan 5 2023 10:22AM,254331,19,"GUSA Granules USA, Inc.",1.0,0.0
2,Jan 5 2023 10:11AM,254330,10,"Citieffe, Inc.",1.0,0.0
3,Jan 5 2023 9:01AM,254327,20,"Adare Pharmaceuticals, Inc.",1.0,0.0
4,Jan 5 2023 8:54AM,254324,19,"GUSA Granules USA, Inc.",0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,508631,1.0,1.0
12,254332,1.0,0.0
15,762884,39.0,0.0
16,508586,2.0,0.0
19,1525867,72.0,1.0
20,254327,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,508631,1.0,1.0
1,12,254332,1.0,0.0
2,15,762884,39.0,0.0
3,16,508586,2.0,0.0
4,19,1525867,72.0,1.0
5,20,254327,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,1.0,1.0
1,12,1.0,0.0
2,15,39.0,0.0
3,16,2.0,0.0
4,19,72.0,1.0
5,20,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1.0
1,12,Released,1.0
2,15,Released,39.0
3,16,Released,2.0
4,19,Released,72.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Executing,1.0,0.0,0.0,0.0,1.0,0.0
Released,1.0,1.0,39.0,2.0,72.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Executing,1.0,0.0,0.0,0.0,1.0,0.0
1,Released,1.0,1.0,39.0,2.0,72.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Executing,1.0,0.0,0.0,0.0,1.0,0.0
1,Released,1.0,1.0,39.0,2.0,72.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()